In [1]:
import pandas as pd
from fbprophet import Prophet
import matplotlib.pyplot as plt
import os

In [2]:
#Population values
population = {
    "Chiapas_2016-2017.csv": 5217908,
    "Colima_2016-2017.csv": 711235,
    "Guerrero_2016-2017.csv": 3533251,
    "Hidalgo_2016-2017.csv": 2858359,
    "NuevoLeon_2016-2017.csv": 5119504,
    "Oaxaca_2016-2017.csv": 3967889,
    "QuintanaRoo_2016-2017.csv": 1501562,
    "Tabasco_2016-2017.csv" : 2395272,
    "Veracruz_2016-2017.csv" : 8112505,
    "Yucatan_2016-2017.csv" : 2097175,
    
    "casanare_2016-2017.csv" : 356438,
    "cordoba_2016-2017.csv" : 1709603,
    "cundinamarca_2016-2017.csv" : 2680041,
    "huila_2016-2017.csv" : 1154804,
    "meta_2016-2017.csv" : 961292,
    "santander_2016-2017.csv" : 2061095,
    "santander_norte_2016-2017.csv" : 1355723,
    "tolima_2016-2017.csv" : 1408274,
    "valle_cauca_2016-2017.csv" : 4613377,
    
    "Alagoas_2016-2017.csv": 3375823,
    "Bahia_2016-2017.csv": 15344447,
    "Ceara_2016-2017.csv": 9020460,
    "Goias_2016-2017.csv": 6778772,
    "Maranhao_2016-2017.csv": 7000229,
    "MatoGrosso_2016-2017.csv": 3344544,
    "MinasGerais_2016-2017.csv": 21119536,
    "Para_2016-2017.csv": 8366628,
    "RioDeJaneiro_2016-2017.csv": 16718956,
    "SaoPaulo_2016-2017.csv": 45094866,
}

In [3]:
def formatFilename(filename):
    return filename.replace(".csv", "")

In [7]:
for country in ["Mexico", "Brazil", "Colombia"]:
    folder = "../../data/{}/processed_data".format(country)
    files = os.listdir(folder)
    for file in files:
        dataset = pd.read_csv("{}/{}".format(folder, file))
        state = file
        
        #Preprocess data
        scale = population[state]
        dataset[["Searches"]] /= 100
        dataset[["Cases"]] = dataset[["Cases"]].apply(lambda x: x*100000/scale, axis=1)
        
        if(not os.path.isdir("{}/{}".format(country, file))):
            os.mkdir("{}/{}".format(country, file))
    
        formattedFilename = formatFilename(file)
        
        df = pd.DataFrame()
        df["ds"] = dataset["Date"]
        df["y"]  = dataset["Cases"]
        df["Searches"] = dataset["Searches"]
        
        model = Prophet()
        model.add_regressor('Searches')
        model.fit(df)
        
        test_index = 52
        pred = model.predict(df[test_index:]) #Predict second year's cases
        
        plot_df = pd.DataFrame(pred["ds"][test_index:])
        plot_df.rename(index=str, columns={"ds": "Date"}, inplace=True)
        plot_df.set_index("Date", inplace=True)
        
        #Rescale to normal
        plot_df["Predicted"] = pred["yhat"].values * (scale / 100000)
        plot_df["Observed"] = dataset["Cases"].values[test_index:] * (scale / 100000)
        
        plt.clf()
        cols = ["Observed", "Predicted"]
        colors = ['#2962FF', '#D500F9']
        plot_df[cols].plot(
            figsize=(10, 10),
            grid=True,
            color=colors
        )
        ax = plt.gca()
        ax.set_facecolor((0.9, 0.9, 0.9, 0.7))
        plt.title("Prophet Model\n{}".format(formattedFilename))
        plt.xlabel("Date")
        plt.ylabel("ZIKV Cases")
        plt.grid(linestyle='dashed', linewidth=1.5)
        fig = plt.gcf()
        fig.savefig("{}/{}/Prophet-{}.png".format(country, file, formattedFilename))
        plt.close("all")
        
        plot_df["error"] = plot_df["Predicted"] - plot_df["Observed"]
        pred.to_csv("{}/{}/ProphetOutput-{}.csv".format(country, file, formattedFilename))
        plot_df.to_csv("{}/{}/Prophet-{}.csv".format(country, file, formattedFilename))

INFO:fbprophet.forecaster:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling yearly seasonality. Run prophet with yearly_seasonal